In [15]:
# imports
import requests
import json
import csv
import time
import datetime
from string import Template
from Config import *
import sqlite3
from sqlite3 import Error

In [16]:
# Variables
headers = {"Authorization": API_TOKEN}
min_stars = 237
max_stars = 10000
last_activity = 90 # within the last __ days
created = 364 * 4 # within the last __ days
min_pull_num = 0 # amount of pull requests a repository needs
watchers = 0 # amount of watchers a repository needs

In [17]:
# Builds the query filter string compatible to github
def query_filter( min_stars, last_activity, created ):
    date_last_act = datetime.datetime.now() - datetime.timedelta( days=last_activity )
    date_created = datetime.datetime.now() - datetime.timedelta( days=created )
    stars = f'{min_stars}..1000'

    return f'is:public archived:false fork:false stars:{stars} pushed:20{date_last_act:%y-%m-%d}..* created:20{date_created:%y-%m-%d}..* sort:stars-asc'

In [18]:
# Funtion that uses requests.post to make the API call
def run_query(query, variables):
    request = requests.post('https://api.github.com/graphql', 
                            json={'query': query, 'variables' : variables}, headers=headers)
    #if request.status_code == 200:
        
    if 'Retry-After' in request.headers:  # reached retry limit
        print(f'[WAITING] for {request.headers["Retry-After"]} seconds before continuing...', end=' ')
        time.sleep(int(request.headers['Retry-After']))

    return request.json()
    #else:
    #    raise Exception(f'ERROR [{request.status_code}]: Query failed to execute...\nRESPONSE: {request.text}')

In [19]:
end_cursor = ""
query_string = query_filter( min_stars, last_activity, created)
variables = {
    "query_string" : query_string,
    "end_cursor" : end_cursor,
    "repositoriesPerPage" : 1 # from 1 to 100
}

In [20]:
# setups the github graphql queries
def setup_query( variables, end_cursor ) :
    variables['end_cursor'] = end_cursor
    query = '''
    query($query_string : String!, $repositoriesPerPage : Int) {
        rateLimit{
            cost
            remaining
            resetAt
        }
        search(query: $query_string, type: REPOSITORY, first: $repositoriesPerPage) {
            pageInfo {
                hasNextPage
                endCursor
            }
            repositoryCount
            nodes {
                ... on Repository {
                    id
                    name
                    owner {
                        login
                    }
                    createdAt
                    isMirror
                    isFork
                    diskUsage
                    primaryLanguage {
                        name
                    }
                    languages(first:100) {
                        totalCount
                        nodes {
                            name
                        }
                    }
                    contributors : mentionableUsers {
                        totalCount
                    }
                    watchers {
                        totalCount
                    }
                    stargazers {
                        totalCount
                    }
                    forkCount
                    issues {
                        totalCount
                    }
                    commits : defaultBranchRef {
                        target {
                            ... on Commit {
                                history {
                                    totalCount
                                }
                            }
                        }
                    }
                    pullRequests {
                        totalCount
                    }
                    releases {
                        totalCount
                    }
                    licenseInfo {
                        id
                        name
                        pseudoLicense
                    }
                    url
                }
            }
        }
    }'''
    query2 = '''
    query($query_string : String!, $end_cursor : String, $repositoriesPerPage : Int) {
        rateLimit{
            cost
            remaining
            resetAt
        }
        search(query: $query_string, type: REPOSITORY, first: $repositoriesPerPage, after: $end_cursor) {
            pageInfo {
                hasNextPage
                endCursor
            }
            repositoryCount
            nodes {
                ... on Repository {
                    owner {
                        login
                    }
                    id
                    name
                    description
                    createdAt
                    pushedAt
                    isDisabled
                    isMirror
                    isFork
                    isLocked
                    diskUsage
                    primaryLanguage {
                        name
                    }
                    languages(first:100) {
                        totalCount
                        nodes {
                            name
                        }
                    }
                    contributors : mentionableUsers {
                        totalCount
                    }
                    watchers {
                        totalCount
                    }
                    stargazers {
                        totalCount
                    }
                    forkCount
                    issues {
                        totalCount
                    }
                    commits : defaultBranchRef {
                        target {
                            ... on Commit {
                                history {
                                    totalCount
                                }
                            }
                        }
                    }
                    pullRequests {
                        totalCount
                    }
                    releases {
                        totalCount
                    }
                    licenseInfo {
                        id
                        name
                        pseudoLicense
                    }
                    url
                }
            }
        }
    }'''
    if(end_cursor == "") : return (query, variables)
    else : return (query2, variables)

In [21]:
# Runs the query and iterates through all pages of repositories
def iterate_queries( conn, create_dict_method ):

    end_cursor = ""
    end_cursor_string = ""
    hasNextPage = True
    index = 0
    
    ai = 1 # slow start: 1, 2, 4, 8 (max)
    md = 0.5
    
    print("[WORKING] Running script to collect all repositories. ")
    while( hasNextPage ):
        print("[WORKING] On page " + str(index))
        query = setup_query( variables, end_cursor )
        result = run_query( query[0], query[1] )
        #print(json.dumps(result))
        
        if 'errors' in result:
            if 'timeout' in result['errors'][0]['message']:  # reached timeout
                variables['repositoriesPerPage'] = int(max(1, variables['repositoriesPerPage'] * md))  # using AIMD
                ai = 1  # resetting slow start
                print('[WORKING] Timeout! - Ressting page size to : ' + 
                      str(variables['repositoriesPerPage']))
            else:  # some unexpected error.
                print(result['errors'])
                exit(1)
        
        if 'data' in result and result['data']:
            
            #print(json.dumps(result))
                      
            insert_repositories( conn, result, create_dict_method )

            if(result['data']['search']['nodes']):
                stars = result['data']['search']['nodes'][-1]['stargazers']['totalCount']
            
            try:
                # if there is a next page, update the endcursor string and continue loop
                if( result["data"]["search"]["pageInfo"]["hasNextPage"] ):
                    end_cursor = result["data"]["search"]["pageInfo"]["endCursor"]
                    
                    variables['repositoriesPerPage'] = min(100, variables['repositoriesPerPage'] + ai)  # using AIMD
                    print("[WORKING] New reposPerPage set to : " + str(variables['repositoriesPerPage']))
                    ai = min(8, ai * 2)  # slow start
                    #end_cursor_string = f', after:"{end_cursor}"'
                else:
                    if(result['data']['search']['repositoryCount'] > 1000) :
                        print('[WORKING] We reached the limit of 1,000 repositories.')
                        variables['query_string'] = query_filter(stars, last_activity, created)  # some overlap to accommodate changes in stars
                        print('[WORKING] New selected star value is : ' + str(stars))
                        end_cursor = ""
                    else :
                        hasNextPage = False
                        end_cursor = ""
                        print(json.dumps(result))
            except KeyError:
                print("[WORKING] No next page. ")
                break 

            index += 1
        time.sleep(1)

In [22]:
def create_repo_tuple( node ) :
    commits = 0
    if(node["commits"]["target"]["history"]["totalCount"] != None) :
        commits = node["commits"]["target"]["history"]["totalCount"]
        
    primaryLanguage = "null"
    if(node["primaryLanguage"] != None) :
        primaryLanguage = node["primaryLanguage"]["name"]
        
    license_id = "0"
    license_name = "null"
    pseudoLicense = False
    if(node["licenseInfo"] != None) :
        license_id = node["licenseInfo"]["id"]
        license_name = node["licenseInfo"]["name"]
        pseudoLicense = node["licenseInfo"]["pseudoLicense"]
        
    return (
    node["id"], 
    node["name"], 
    node["owner"]["login"], 
    node["createdAt"], 
    node["isMirror"], 
    node["isFork"], 
    node["diskUsage"], 
    primaryLanguage,
    node["contributors"]["totalCount"], 
    node["watchers"]["totalCount"], 
    node["stargazers"]["totalCount"], 
    node["forkCount"], 
    node["issues"]["totalCount"], 
    commits, 
    node["pullRequests"]["totalCount"], 
    node["releases"]["totalCount"],
    license_id, 
    license_name, 
    pseudoLicense, 
    node["url"]
    )

In [23]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
    return conn

In [24]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [25]:
# insert repository tuples into the sqlite database
def insert_repositories( conn, result, create_dict_method ):
    
    data_list = []
    nodes = result["data"]["search"]["nodes"]
    for node in nodes:
        data_list.append( create_dict_method( node ) )
    
    """
    Create a new repository into the repository table
    :param conn:
    :param repository:
    :return: project id
    """
    sql = """INSERT or IGNORE INTO repositories (id, name, owner, start_date, isMirror, isFork, diskUsage, 
                     primaryLanguage, numContributors, watchers, stars, forks, issues, commits, 
                     pullRequests, releases, license_id, license_name, pseudoLicense, url) VALUES
                     (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    
    cur = conn.cursor()
    cur.executemany(sql, data_list)
    conn.commit()
    print("[SUCCESS] Inserted repos into the database. ")
    cur.close()
    return cur.lastrowid

In [26]:
# name of the sqlite file
database = r".\databases\github_data.db"

# create a database connection
conn = create_connection(database)

In [27]:
# sql for creating the repos table
sql_create_repos_table = """CREATE TABLE IF NOT EXISTS repositories (
                                        id text PRIMARY KEY NOT NULL,
                                        name text NOT NULL,
                                        owner text NOT NULL,
                                        start_date text NOT NULL,
                                        isMirror boolean NOT NULL,
                                        isFork boolean NOT NULL,
                                        diskUsage int NOT NULL,
                                        primaryLanguage text NOT NULL,
                                        numContributors int NOT NULL,
                                        watchers int NOT NULL,
                                        stars int NOT NULL,
                                        forks int NOT NULL,
                                        issues int NOT NULL,
                                        commits int NOT NULL,
                                        pullRequests int NOT NULL,
                                        releases int NOT NULL,
                                        license_id text NOT NULL,
                                        license_name text NOT NULL,
                                        pseudoLicense boolean NOT NULL,
                                        url text NOT NULL
                                    ); """

with conn:
    create_table(conn, sql_create_repos_table)

In [28]:
# when conn is valid
with conn:
    # run query to find all repositories
    # pass in the repository tuple builder
    # insert the tuple into the database
    iterate_queries( conn, create_repo_tuple )

[WORKING] Running script to collect all repositories. 
[WORKING] On page 0
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 2
[WORKING] On page 1
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 4
[WORKING] On page 2
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 8
[WORKING] On page 3
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 16
[WORKING] On page 4
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 24
[WORKING] On page 5
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 32
[WORKING] On page 6
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 40
[WORKING] On page 7
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 48
[WORKING] On page 8
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 56
[WORKING] On

[WORKING] On page 29
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 67
[WORKING] On page 30
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 75
[WORKING] On page 31
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 83
[WORKING] On page 32
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 91
[WORKING] On page 33
[WORKING] Timeout! - Ressting page size to : 45
[WORKING] On page 33
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 46
[WORKING] On page 34
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 48
[WORKING] On page 35
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 52
[WORKING] On page 36
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 60
[WORKING] On page 37
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPag

[WORKING] On page 99
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 51
[WORKING] On page 100
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 53
[WORKING] On page 101
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 57
[WORKING] On page 102
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 65
[WORKING] On page 103
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 73
[WORKING] On page 104
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 81
[WORKING] On page 105
[WORKING] Timeout! - Ressting page size to : 40
[WORKING] On page 105
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 41
[WORKING] On page 106
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 43
[WORKING] On page 107
[SUCCESS] Inserted repos into the database. 
[WORKING] New re

[WORKING] On page 169
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 71
[WORKING] On page 170
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 79
[WORKING] On page 171
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 87
[WORKING] On page 172
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 95
[WORKING] On page 173
[WORKING] Timeout! - Ressting page size to : 47
[WORKING] On page 173
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 48
[WORKING] On page 174
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 50
[WORKING] On page 175
[SUCCESS] Inserted repos into the database. 
[WORKING] We reached the limit of 1,000 repositories.
[WORKING] New selected star value is : 504
[WORKING] On page 176
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 54
[WORKING] On page 177
[S

[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 52
[WORKING] On page 239
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 60
[WORKING] On page 240
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 68
[WORKING] On page 241
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 76
[WORKING] On page 242
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 84
[WORKING] On page 243
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 92
[WORKING] On page 244
[WORKING] Timeout! - Ressting page size to : 46
[WORKING] On page 244
[SUCCESS] Inserted repos into the database. 
[WORKING] New reposPerPage set to : 47
[WORKING] On page 245
[SUCCESS] Inserted repos into the database. 
[WORKING] We reached the limit of 1,000 repositories.
[WORKING] New selected star value is : 827
[WORKING] On page 246
[SUCCESS] Inserted repos